In [7]:
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms,models
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score

In [8]:
APTOS_ROOT = r"C:\DCU\Thesis\data\APTOS"

TRAIN_IMG_DIR = os.path.join(APTOS_ROOT, "train_images")
VAL_IMG_DIR   = os.path.join(APTOS_ROOT, "val_images")

TRAIN_CSV = os.path.join(APTOS_ROOT, "train_1.csv")
VAL_CSV   = os.path.join(APTOS_ROOT, "valid.csv")

In [9]:
train_data = pd.read_csv(TRAIN_CSV)
val_data = pd.read_csv(VAL_CSV)

train_data.head()
print(train_data['diagnosis'].value_counts())

diagnosis
0    1434
2     808
1     300
4     234
3     154
Name: count, dtype: int64


# TRANSFORMS

In [10]:
IMAGENET_MEAN = (0.485,0.456,0.406)
IMAGENET_STD = (0.229,0.224,0.225)

train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

In [15]:
class APTOSDataset(Dataset):
    def __init__(self, data, img_dir, transform):
        self.data = data.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_id = row["id_code"]
        label = int(row["diagnosis"])

        img_path = os.path.join(self.img_dir, img_id + ".png")
        if not os.path.exists(img_path):
            img_path = os.path.join(self.img_dir, img_id + ".jpg")

        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)

        return image, label

## DATALOADERS

In [21]:
train_ds = APTOSDataset(train_data, TRAIN_IMG_DIR, train_tfms)
val_ds   = APTOSDataset(val_data, VAL_IMG_DIR, val_tfms)

train_loader = DataLoader(
    train_ds, batch_size=32, shuffle=True, pin_memory=True
)

val_loader = DataLoader(
    val_ds, batch_size=64, shuffle=False, pin_memory=True
)


## MODEL

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 5)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)

Using device: cpu


## TRAIN AND EVALUATE

In [27]:
def run_epoch(model,loader,train = True):
    model.train() if train else model.eval()
    all_labels,all_probs,all_preds = [],[],[]
    total_loss = 0
    
    for x,y in loader:
        x,y = x.to(device),y.to(device)
        
        with torch.set_grad_enabled(train):
            logits = model(x)
            loss = criterion(logits,y)
            
            if train:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
        probs = torch.softmax(logits,dim = 1).detach().cpu().numpy()
        preds = np.argmax(probs,axis = 1)
        
        all_probs.append(probs)
        all_preds.append(preds)
        all_labels.append(y.cpu().numpy())
        total_loss += loss.item() * x.size(0)
    
    all_probs = np.concatenate(all_probs)
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    
    acc = accuracy_score(all_labels, all_preds)
    try:
        auc = roc_auc_score(all_labels, all_probs, multi_class="ovr")
    except:
        auc = None

    cm = confusion_matrix(all_labels, all_preds)
    avg_loss = total_loss / len(loader.dataset)

    return avg_loss, acc, auc, cm

In [28]:
EPOCHS = 5

for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc, tr_auc, _ = run_epoch(model, train_loader, train=True)
    va_loss, va_acc, va_auc, va_cm = run_epoch(model, val_loader, train=False)

    print(f"\nEpoch {epoch}/{EPOCHS}")
    print(f"Train: loss={tr_loss:.4f}, acc={tr_acc:.4f}, auc={tr_auc}")
    print(f"Val  : loss={va_loss:.4f}, acc={va_acc:.4f}, auc={va_auc}")
    print("Confusion Matrix:\n", va_cm)


Epoch 1/5
Train: loss=0.7148, acc=0.7464, auc=0.8779154154621027
Val  : loss=0.5976, acc=0.7705, auc=0.914690085327511
Confusion Matrix:
 [[170   0   1   0   1]
 [  2   6  29   0   3]
 [  3   1  85   7   8]
 [  1   0   6   7   8]
 [  0   0  12   2  14]]

Epoch 2/5
Train: loss=0.4715, acc=0.8154, auc=0.9416695198896562
Val  : loss=0.4927, acc=0.8306, auc=0.9413123859324124
Confusion Matrix:
 [[171   1   0   0   0]
 [  3  24  12   0   1]
 [  2   5  83  13   1]
 [  0   1   6  12   3]
 [  0   2   7   5  14]]

Epoch 3/5
Train: loss=0.4274, acc=0.8399, auc=0.9517268879327784
Val  : loss=0.4885, acc=0.7978, auc=0.9456931759919686
Confusion Matrix:
 [[171   1   0   0   0]
 [  2  18  19   0   1]
 [  4   4  95   1   0]
 [  1   0  18   3   0]
 [  0   1  19   3   5]]

Epoch 4/5
Train: loss=0.3631, acc=0.8635, auc=0.9648043471962111
Val  : loss=0.5036, acc=0.8306, auc=0.9400944112061662
Confusion Matrix:
 [[171   1   0   0   0]
 [  0  19  18   0   3]
 [  1   4  89   7   3]
 [  0   0   8   8   6]
 

In [ ]:
os.makedirs("checkpoints", exist_ok=True)
torch.save(model.state_dict(), "checkpoints/aptos_resnet50_baseline.pt")

We can observe that after training and validation there is a rise in accuracy,might be a slight overfit but 